In [1]:
import torch

### I. Simplified self-attention

$$
\text{Inputs}: \mathbf{x} =
\begin{bmatrix}x^0 & x^1 & x^2 & \cdots & x^T\end{bmatrix}
$$

In [2]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^0)
     [0.55, 0.87, 0.66], # journey  (x^1)
     [0.57, 0.85, 0.64], # starts   (x^2)
     [0.22, 0.58, 0.33], # with     (x^3)
     [0.77, 0.25, 0.10], # one      (x^4)
     [0.05, 0.80, 0.55]] # step     (x^5)
)

1. Compute the **attention scores**
$$\mathbf{w} = \begin{bmatrix}w_0 & w_1 & w_2 & \cdots & w_T\end{bmatrix}$$
where $w_i = \mathbf{x} \times x^i$

3. Compute the **context vector**
$$ 
\mathbf{z} = \begin{bmatrix}z_0 & z_1 & z_2 & \cdots & z_T\end{bmatrix}
$$
where $z_i = w_i \times \mathbf{x}$

---

**Example :**<br>
We use a query $q$, for the second input query $q = x^2 = [0.55, 0.87, 0.66]$ corresponding to the word *journey* :<br>
1. $w_2 = \mathbf{x} \times q = [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]$
2. $z_2 = w_2 \times \mathbf{x} = [0.4419, 0.6515, 0.5683]$

In [3]:
# 1. choose an input query (q = x^k)
query = inputs[1]
# 2. compute the attention scores : dot product (w^i = x^i * q)
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

# 3. normalize to get the attention weights
print(attn_scores_2)
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print(attn_weights_2_tmp, "sum=", attn_weights_2_tmp.sum())

# (3bis. we could use softmax normalization instead)
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights_2, "sum=", attn_weights_2.sum())

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656]) sum= tensor(1.0000)
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) sum= tensor(1.)


In [4]:
# 4. compute context vector (z^k = Σw^i * x^i)
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


---

In [5]:
# attention scores (w)
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [6]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [7]:
# normalization to get attention weights
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [8]:
# context vectors (z)
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

---

### II. Adding trainable weights

In [9]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [10]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [13]:
query_2 = x_2 @ W_query
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2, key_2, value_2)

tensor([0.4306, 1.4551]) tensor([0.4433, 1.1419]) tensor([0.3951, 1.0037])


In [17]:
keys = inputs @ W_key
values = inputs @ W_value

print(keys, keys.shape)
print(values, values.shape)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]]) torch.Size([6, 2])
tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]]) torch.Size([6, 2])


---

In [21]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [22]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


![](https://camo.githubusercontent.com/2ac6b95aae5b921331210eb532d2982e836156c19a2778db2247386fe7a7d8a9/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f31362e77656270)

In [24]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


![](https://camo.githubusercontent.com/1c9e89b0210f28f3064c10f70a4e6d327cd69e8228dcfe0cbd7de47dea3dae37/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f31372e77656270)

In [30]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

---

In [31]:
import torch.nn as nn

In [44]:
class SelfAttention(nn.Module):
    def __init__(self, d_in, d_out, qkv_biases=False):
        super().__init__()
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_biases) # nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_biases) # nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_biases) # nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = self.W_key(x) # x @ self.W_query
        queries = self.W_query(x) # x @ self.W_query
        values = self.W_value(x) # x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [47]:
torch.manual_seed(789)
sa = SelfAttention(d_in, d_out)
print(sa(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


---

### III. Causal Attention

![](https://camo.githubusercontent.com/ae6a1857af914fbb7d57da177ce6bff4b57dbebe3bd11395855b3315ae13d1e1/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f31392e77656270)
![](https://camo.githubusercontent.com/085c511ba76dafdd9bc7d9fc8e15fdf24770dfa7d8061cc2441675411051c5f0/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f32302e77656270)

In [55]:
# 1)
queries = sa.W_query(inputs)
keys = sa.W_key(inputs)
attn_scores = queries @ keys.T
# 2)
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**.5, dim=-1)

attn_weights

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [56]:
# 3)
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))

print(mask_simple)

masked_simple = attn_weights * mask_simple

masked_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

In [58]:
# 4)
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums

masked_simple_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)

---

In [66]:
# another method without step 2)
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(mask)

masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
masked

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])


tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)

In [67]:
attn_weights = torch.softmax(masked / keys.shape[-1]**.5, dim=-1)

attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

---

![](https://camo.githubusercontent.com/524540987cbb8371fa7d5cd8f1816816b8d17ddea93ad9ecf2cd2a2e32c5c7bd/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f32322e77656270)

In [74]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(.5)
example = torch.ones((6,6))

# this will also scale the matrix by 1/.5 = 2
# to compensate for the reduction in active elements
dropout(example)

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])

In [72]:
torch.manual_seed(123)
dropout(attn_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)

---

In [77]:
batch = torch.stack([inputs, inputs], dim=0)

(batch, batch.shape)

(tensor([[[0.4300, 0.1500, 0.8900],
          [0.5500, 0.8700, 0.6600],
          [0.5700, 0.8500, 0.6400],
          [0.2200, 0.5800, 0.3300],
          [0.7700, 0.2500, 0.1000],
          [0.0500, 0.8000, 0.5500]],
 
         [[0.4300, 0.1500, 0.8900],
          [0.5500, 0.8700, 0.6600],
          [0.5700, 0.8500, 0.6400],
          [0.2200, 0.5800, 0.3300],
          [0.7700, 0.2500, 0.1000],
          [0.0500, 0.8000, 0.5500]]]),
 torch.Size([2, 6, 3]))

In [83]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_biases=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_biases)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_biases)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_biases)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**.5, dim=-1)
        attn_weights = dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [86]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)

context_vecs, context_vecs.shape

(tensor([[[-0.8158, -0.1411],
          [-0.6920, -0.0972],
          [-0.4050, -0.1201],
          [-0.6902, -0.0969],
          [-0.5199, -0.0440],
          [-0.1417, -0.0505]],
 
         [[-0.7938, -0.2379],
          [-0.7858, -0.1145],
          [-0.3969,  0.0037],
          [-0.7704, -0.2374],
          [-0.7801, -0.1107],
          [-0.6749, -0.0984]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 6, 2]))

---

### IV. Multi-head attention

In [87]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_biases=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_biases) for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat(
            [head(x) for head in self.heads],
        dim=-1)

![](https://camo.githubusercontent.com/a8c7145749f9a478fd5f58ea4fb4f3d61516e6a54920099acb0fc1eb7e28d62c/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f32352e77656270)

In [96]:
torch.manual_seed(123)
context_length = batch.shape[1] # number of tokens (~of words)
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

context_vecs, context_vecs.shape

(tensor([[[-0.6357, -0.1859,  0.2343,  0.1723],
          [-0.5118, -0.1684,  0.4834,  0.3947],
          [-0.7776, -0.2389,  0.5966,  0.4618],
          [-0.1479,  0.0725,  0.5227,  0.3902],
          [-0.5422, -0.1692,  0.4725,  0.3869],
          [-0.9130, -0.2881,  0.8947,  0.5938]],
 
         [[-0.5025, -0.1652,  0.2997,  0.1348],
          [-0.2952, -0.1008,  0.7681,  0.4786],
          [-0.8158, -0.1470,  0.3625,  0.2897],
          [-0.5271, -0.1706,  0.7440,  0.5439],
          [-0.6787, -0.0962,  0.5175,  0.3862],
          [-0.4257, -0.1539,  0.6311,  0.4446]]], grad_fn=<CatBackward0>),
 torch.Size([2, 6, 4]))

---

In [103]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

Reshaping (`.view`) and transposing (`.transpose`) are used to perform a single matrix multiplication instead of multiples.

To do so, the matrices are first stacked then split then flattened.

![](https://camo.githubusercontent.com/96c1092c9ded98192988eae5300784c871833c9ef736997edfa0c97189076175/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830335f636f6d707265737365642f32362e77656270)

In [104]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

context_vecs, context_vecs.shape

(tensor([[[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]],
 
         [[0.3190, 0.4858],
          [0.2943, 0.3897],
          [0.2856, 0.3593],
          [0.2693, 0.3873],
          [0.2639, 0.3928],
          [0.2575, 0.4028]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 6, 2]))